In [0]:
from keras.datasets import imdb
from keras.preprocessing import sequence
import tensorflow as tf
import os
import numpy as np

In [0]:
vocab_size = 100000
maxlen = 250
batch_size = 64

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocab_size)

In [0]:
X_train = sequence.pad_sequences(X_train, maxlen)
X_test = sequence.pad_sequences(X_test, maxlen)

In [0]:
model = tf.keras.Sequential([
                             tf.keras.layers.Embedding(vocab_size,32),
                             tf.keras.layers.LSTM(32),
                             tf.keras.layers.Dense(1, activation="sigmoid")
])

In [66]:
model.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['acc'])
histroy = model.fit(X_train, y_train, epochs=5, validation_split=0.1)

Epoch 1/5
704/704 [==============================] - 20s 28ms/step - loss: 0.0946 - acc: 0.9702 - val_loss: 0.3329 - val_acc: 0.8896
Epoch 2/5
704/704 [==============================] - 19s 27ms/step - loss: 0.0787 - acc: 0.9745 - val_loss: 0.3297 - val_acc: 0.8896
Epoch 3/5
704/704 [==============================] - 18s 26ms/step - loss: 0.0665 - acc: 0.9783 - val_loss: 0.4239 - val_acc: 0.8868
Epoch 4/5
704/704 [==============================] - 19s 26ms/step - loss: 0.0595 - acc: 0.9812 - val_loss: 0.3607 - val_acc: 0.8824
Epoch 5/5
704/704 [==============================] - 19s 27ms/step - loss: 0.0512 - acc: 0.9839 - val_loss: 0.3913 - val_acc: 0.8900


In [67]:
results = model.evaluate(X_test, y_test)
print(results)
model.save("sentiment-RNN-imdb.h5")

782/782 [==============================] - 5s 7ms/step - loss: 0.5328 - acc: 0.8504
[0.5327765345573425, 0.8504400253295898]


In [0]:
model = tf.keras.models.load_model("sentiment-RNN-imdb.h5")

word_index = imdb.get_word_index()
rev_word_index = {value: key for (key, value) in word_index.items()}

def encode_text(text): return sequence.pad_sequences( [ [word_index[word] if word in word_index else 0 for word in (tf.keras.preprocessing.text.text_to_word_sequence(text))] ], maxlen)[0]

def decode_integer(integer): return ' '.join(map(str, [rev_word_index[num] for num in integer if num!= 0]))

def prediction(text):
  pred = np.zeros((1, 250))
  pred[0] = encode_text(text)
  pred_val = model.predict(pred)
  return (" Positive " if (pred_val) >= 0.5  else " Negative "), pred_val


In [82]:
print(prediction(str(input("Enter a movie review: "))))

Enter a movie review: a must watch movie
(' Negative ', array([[0.17310469]], dtype=float32))
